In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from cnn_model import CNN  # Your CNN model
from data_utils import get_cnn_loader  # Your loader for MNIST/EMNIST


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CNN(num_classes=10)  # or 47 for EMNIST
model.load_state_dict(torch.load("cnn_mnist.pt", map_location=device))
model.to(device)
model.eval()


In [ ]:
test_loader = get_cnn_loader(dataset_name="MNIST", batch_size=16, train=False)


In [ ]:
classes = [str(i) for i in range(10)]  # Change if using EMNIST (letters)

def show_predictions(loader, model, num_batches=1):
    for i, (images, labels) in enumerate(loader):
        if i >= num_batches:
            break
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        images = images.cpu().numpy()
        preds = preds.cpu().numpy()
        labels = labels.cpu().numpy()

        plt.figure(figsize=(12, 3))
        for j in range(len(images)):
            plt.subplot(1, len(images), j+1)
            plt.imshow(images[j][0], cmap="gray")
            plt.title(f"Pred: {classes[preds[j]]}\nTrue: {classes[labels[j]]}")
            plt.axis("off")
        plt.show()

show_predictions(test_loader, model)


In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
